# Part 4: SHAP 분석 및 비즈니스 가치 평가

## 🎯 목표

Part 3 v3에서 선정된 최종 모델의 예측 근거를 **SHAP (SHapley Additive exPlanations)**으로 해석하고, 비즈니스 의사결정에 활용 가능한 인사이트를 도출합니다.

---

## 📋 선행 조건

### Part 3 v3 출력 파일

- `발표_Part3_v3_최종모델.pkl`
- `발표_Part3_v3_임계값.pkl`
- `발표_Part3_v3_결과.pkl`
- `domain_based_features_완전판.csv`

### 데이터 규모

- **기업 수**: 50,105개
- **Feature 수**: 27개 (Part 2에서 선택된 도메인 기반 특성)
- **부도율**: ~1.5%
- **Train/Val/Test Split**: 60% / 20% / 20%

---

## 섹션 0: 환경 설정 및 데이터 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import joblib
import warnings
import json
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

warnings.filterwarnings('ignore')

# 한글 폰트 설정
import platform
if platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:
    plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

# 시각화 스타일
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("✅ 라이브러리 로딩 완료")
print(f"   - SHAP 버전: {shap.__version__}")
print(f"   - Python 플랫폼: {platform.system()}")

In [ ]:
# Part 3 v3 출력 로드
PROCESSED_DIR = '../data/processed'

try:
    final_model = joblib.load(os.path.join(PROCESSED_DIR, '발표_Part3_v3_최종모델.pkl'))
    thresholds = joblib.load(os.path.join(PROCESSED_DIR, '발표_Part3_v3_임계값.pkl'))
    results = joblib.load(os.path.join(PROCESSED_DIR, '발표_Part3_v3_결과.pkl'))
    
    print("✅ Part 3 v3 모델 로드 완료")
    print(f"   - 최종 모델: {results['model_name']}")
    print(f"   - Test PR-AUC: {results['test_pr_auc']:.4f}")
    print(f"   - Test Recall: {results['test_recall']:.2%}")
    print(f"   - Test F2-Score: {results['test_f2']:.4f}")
    print(f"\n✅ 임계값 로드 완료")
    print(f"   - 선택된 임계값: {thresholds['selected']:.4f}")
    print(f"   - Red 임계값: {thresholds['red']:.4f}")
    print(f"   - Yellow 임계값: {thresholds['yellow']:.4f}")
except FileNotFoundError as e:
    print(f"❌ 오류: Part 3 v3 출력 파일을 찾을 수 없습니다.")
    print(f"   - {e}")
    print(f"\n먼저 '발표_Part3_v3_완전판.ipynb'를 실행해주세요.")
    raise

In [ ]:
# Feature 데이터 로드
FEATURES_PATH = '../data/features/domain_based_features_완전판.csv'

try:
    features_df = pd.read_csv(FEATURES_PATH, encoding='utf-8')
    print(f"✅ Feature 데이터 로드 완료")
    print(f"   - 데이터 크기: {features_df.shape}")
    print(f"   - 기업 수: {len(features_df):,}개")
except FileNotFoundError:
    print(f"❌ 오류: {FEATURES_PATH} 파일을 찾을 수 없습니다.")
    print(f"\n먼저 '발표_Part2_특성선택.ipynb'를 실행해주세요.")
    raise

# 타겟 분리
TARGET_COL = '모형개발용Performance(향후1년내부도여부)'
X = features_df.drop(columns=[TARGET_COL])
y = features_df[TARGET_COL]

print(f"\n✅ 타겟 변수 분리 완료")
print(f"   - Feature 수: {X.shape[1]}개")
print(f"   - 부도 기업: {y.sum():,}개 ({y.mean():.2%})")
print(f"   - 정상 기업: {(~y.astype(bool)).sum():,}개 ({(~y.astype(bool)).mean():.2%})")

In [ ]:
# 동일한 3-Way Split (Part 3와 동일한 random_state 사용)
RANDOM_STATE = 42

# Train/Temp Split (80/20)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

# Train/Val Split (60/20)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=RANDOM_STATE
)

print("✅ 데이터 분할 완료 (Part 3와 동일)")
print(f"\nTrain Set:")
print(f"  - 크기: {len(X_train):,}개")
print(f"  - 부도율: {y_train.mean():.2%}")
print(f"\nValidation Set:")
print(f"  - 크기: {len(X_val):,}개")
print(f"  - 부도율: {y_val.mean():.2%}")
print(f"\nTest Set:")
print(f"  - 크기: {len(X_test):,}개")
print(f"  - 부도율: {y_test.mean():.2%}")

---

## 섹션 1: SHAP TreeExplainer 초기화

### SHAP란?

- **Shapley Value** 기반 모델 해석 방법 (게임 이론에서 유래)
- 각 Feature가 **개별 예측에 기여한 정도**를 정량화
- **양수**: 부도 위험 증가 / **음수**: 부도 위험 감소
- **공정한 분배**: 모든 Feature의 SHAP 값 합 = 예측값 - 기준값

### TreeExplainer

- 트리 기반 모델 (XGBoost, LightGBM, CatBoost) 전용
- 정확하고 빠른 SHAP 값 계산
- KernelExplainer보다 수백 배 빠름

In [ ]:
# TreeExplainer 초기화
print("SHAP TreeExplainer 초기화 중...")

# 파이프라인에서 classifier 추출
classifier = final_model.named_steps['classifier']
print(f"✅ 분류기 타입: {type(classifier).__name__}")

# TreeExplainer 생성
explainer = shap.TreeExplainer(classifier)
print(f"✅ TreeExplainer 초기화 완료")
print(f"   - 기준값 (Expected Value): {explainer.expected_value:.4f}")

In [ ]:
# 전처리된 데이터 준비
print("데이터 전처리 중...")

# 파이프라인의 전처리 단계만 적용 (classifier 제외)
preprocessor = final_model[:-1]
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

print(f"✅ 전처리 완료")
print(f"   - Train Shape: {X_train_preprocessed.shape}")
print(f"   - Test Shape: {X_test_preprocessed.shape}")

In [ ]:
# SHAP values 계산 (Test Set)
print("SHAP values 계산 중... (수 분 소요 가능)")

shap_values = explainer.shap_values(X_test_preprocessed)

# 이진 분류 시 shap_values가 [class0, class1] 형태면 class1만 사용
if isinstance(shap_values, list):
    print(f"   - SHAP values는 리스트 형태 (클래스별 분리)")
    print(f"   - Class 0 shape: {shap_values[0].shape}")
    print(f"   - Class 1 shape: {shap_values[1].shape}")
    shap_values = shap_values[1]  # 부도(1) 클래스
    print(f"   - 부도 클래스(1) SHAP values 선택")

print(f"\n✅ SHAP Values 계산 완료")
print(f"   - Shape: {shap_values.shape}")
print(f"   - 샘플 수: {shap_values.shape[0]:,}개")
print(f"   - Feature 수: {shap_values.shape[1]}개")

---

## 섹션 2: Global Feature Importance (Summary Plot)

전체 데이터셋에서 가장 영향력 있는 Feature를 시각화합니다.

### 해석 가이드

- **Y축**: Feature (중요도 순서, 위로 갈수록 중요)
- **X축**: SHAP Value (양수 = 부도 위험↑, 음수 = 부도 위험↓)
- **색상**: Feature 값 (빨강 = 높음, 파랑 = 낮음)
- **점의 분포**: 넓게 퍼질수록 다양한 영향력

In [ ]:
# Summary Plot (Beeswarm)
plt.figure(figsize=(12, 10))
shap.summary_plot(
    shap_values, 
    X_test_preprocessed, 
    feature_names=X.columns.tolist(),
    show=False,
    max_display=20
)
plt.title('SHAP Summary Plot: Global Feature Importance', fontsize=16, pad=20, weight='bold')
plt.xlabel('SHAP Value (부도 위험에 대한 영향)', fontsize=12)
plt.tight_layout()
plt.savefig('../data/processed/발표_Part4_SHAP_Summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Summary Plot 저장 완료: 발표_Part4_SHAP_Summary.png")

### 주요 패턴 해석

1. **상단 Feature들**: 모델의 핵심 의사결정 요소
2. **빨간 점이 오른쪽**: 해당 Feature 값이 높을수록 부도 위험 증가
3. **파란 점이 왼쪽**: 해당 Feature 값이 낮을수록 부도 위험 감소
4. **수평 분포**: Feature와 부도 위험의 비선형 관계 시사

---

## 섹션 3: Top 10 Feature 상세 분석

가장 영향력 있는 10개 Feature의 재무적 의미를 해석합니다.

In [ ]:
# Top 10 Feature 추출
feature_importance = np.abs(shap_values).mean(axis=0)
top10_idx = np.argsort(feature_importance)[-10:][::-1]
top10_features = X.columns[top10_idx]

print("="*80)
print("Top 10 중요 Feature (Mean |SHAP|)")
print("="*80)
for i, feat in enumerate(top10_features, 1):
    importance = feature_importance[top10_idx[i-1]]
    print(f"{i:2d}. {feat:40s} {importance:.4f}")
print("="*80)

In [ ]:
# Bar Plot
plt.figure(figsize=(12, 7))
shap.summary_plot(
    shap_values, 
    X_test_preprocessed, 
    feature_names=X.columns.tolist(),
    plot_type='bar', 
    show=False, 
    max_display=10
)
plt.title('Top 10 Feature Importance (Mean |SHAP|)', fontsize=16, pad=20, weight='bold')
plt.xlabel('평균 절대 SHAP 값', fontsize=12)
plt.tight_layout()
plt.savefig('../data/processed/발표_Part4_Top10_Features.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Top 10 Bar Plot 저장 완료: 발표_Part4_Top10_Features.png")

In [ ]:
# 재무적 의미 해석 (Feature 이름 기반 자동 생성)
print("\n" + "="*80)
print("Top 10 Feature 재무적 의미")
print("="*80)

# Feature 해석 매핑 (실제 Feature 이름에 맞게 조정 필요)
feature_interpretation = {
    '자본잠식도': ('자본 대비 누적 손실', '높을수록 위험↑ (자본 기반 붕괴)'),
    '이자보상배율': ('영업이익으로 이자 커버 능력', '낮을수록 위험↑ (이자 지급 불가)'),
    '유동비율': ('단기 부채 상환 능력', '낮을수록 위험↑ (유동성 위기)'),
    '당좌비율': ('즉각 현금화 가능 자산 비율', '낮을수록 위험↑ (긴급 유동성 부족)'),
    '부채비율': ('자본 대비 부채 규모', '높을수록 위험↑ (과도한 레버리지)'),
    '총자산이익률(ROA)': ('자산 효율성', '낮을수록 위험↑ (수익성 악화)'),
    '매출채권회전율': ('매출채권 회수 속도', '낮을수록 위험↑ (현금 회수 지연)'),
    '재고자산회전율': ('재고 판매 효율', '낮을수록 위험↑ (재고 적체)'),
}

for i, feat in enumerate(top10_features, 1):
    if feat in feature_interpretation:
        meaning, risk = feature_interpretation[feat]
        print(f"{i:2d}. {feat}")
        print(f"    의미: {meaning}")
        print(f"    위험: {risk}")
    else:
        print(f"{i:2d}. {feat}")
        print(f"    (Feature 이름 기반 해석 추가 필요)")
    print()

print("="*80)

---

## 섹션 4: SHAP Dependence Plot (개별 Feature 분석)

Top 3 Feature의 비선형 관계를 시각화합니다.

### 해석 가이드

- **X축**: Feature 값
- **Y축**: SHAP Value (부도 위험에 대한 기여도)
- **색상**: 상호작용 Feature (자동 선택)
- **패턴**: 비선형 관계, 임계값 효과, 상호작용 확인

In [ ]:
# Top 3 Feature에 대한 Dependence Plot
top3_features = top10_features[:3]

fig, axes = plt.subplots(1, 3, figsize=(20, 5))

for i, feat in enumerate(top3_features):
    feat_idx = list(X.columns).index(feat)
    shap.dependence_plot(
        feat_idx, 
        shap_values, 
        X_test_preprocessed,
        feature_names=X.columns.tolist(), 
        ax=axes[i], 
        show=False
    )
    axes[i].set_title(f'{feat}', fontsize=14, weight='bold')
    axes[i].set_xlabel('Feature 값', fontsize=11)
    axes[i].set_ylabel('SHAP Value', fontsize=11)

plt.suptitle('Top 3 Feature Dependence Plot: 비선형 관계 분석', 
             fontsize=16, weight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../data/processed/발표_Part4_Dependence_Plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Dependence Plot 저장 완료: 발표_Part4_Dependence_Plot.png")

### 주요 패턴

1. **선형 vs 비선형**: 직선/곡선 패턴으로 관계 유형 파악
2. **임계값 효과**: 특정 값을 기준으로 급격한 변화
3. **상호작용**: 색상 그라데이션이 뚜렷하면 다른 Feature와의 상호작용 강함
4. **이상치**: 극단값에서의 SHAP 값 변화

---

## 섹션 5: 개별 기업 사례 분석 (Waterfall Plot)

부도 기업 1개, 정상 기업 1개의 예측 근거를 상세히 시각화합니다.

### Waterfall Plot 해석

- **Base Value**: 전체 평균 예측값 (모든 기업의 평균 부도 확률)
- **화살표**: 각 Feature가 예측값을 증가/감소시키는 정도
- **빨간색**: 부도 위험 증가
- **파란색**: 부도 위험 감소
- **최종값 f(x)**: 해당 기업의 예측 확률

In [ ]:
# 예측 확률 계산
y_test_prob = final_model.predict_proba(X_test)[:, 1]

# 부도 기업 중 확률이 높은 사례
bankrupt_idx = np.where(y_test == 1)[0]
high_risk_idx = bankrupt_idx[np.argsort(y_test_prob[bankrupt_idx])[-1]]

# 정상 기업 중 확률이 낮은 사례
normal_idx = np.where(y_test == 0)[0]
low_risk_idx = normal_idx[np.argsort(y_test_prob[normal_idx])[0]]

print(f"선택된 사례:")
print(f"  - 부도 기업 (고위험): 인덱스 {high_risk_idx}, 예측 확률 {y_test_prob[high_risk_idx]:.2%}")
print(f"  - 정상 기업 (저위험): 인덱스 {low_risk_idx}, 예측 확률 {y_test_prob[low_risk_idx]:.2%}")

In [ ]:
# Expected Value 처리
if isinstance(explainer.expected_value, (list, np.ndarray)):
    expected_value = explainer.expected_value[1]  # 부도 클래스
else:
    expected_value = explainer.expected_value

# Waterfall Plot: 부도 기업
plt.figure(figsize=(12, 8))
shap.waterfall_plot(
    shap.Explanation(
        values=shap_values[high_risk_idx],
        base_values=expected_value,
        data=X_test_preprocessed[high_risk_idx],
        feature_names=X.columns.tolist()
    ),
    show=False,
    max_display=15
)
plt.title(f'부도 기업 예측 근거 (실제: 부도, 예측 확률: {y_test_prob[high_risk_idx]:.2%})', 
          fontsize=14, weight='bold', pad=20)
plt.tight_layout()
plt.savefig('../data/processed/발표_Part4_Waterfall_부도기업.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 부도 기업 Waterfall Plot 저장 완료")

In [ ]:
# Waterfall Plot: 정상 기업
plt.figure(figsize=(12, 8))
shap.waterfall_plot(
    shap.Explanation(
        values=shap_values[low_risk_idx],
        base_values=expected_value,
        data=X_test_preprocessed[low_risk_idx],
        feature_names=X.columns.tolist()
    ),
    show=False,
    max_display=15
)
plt.title(f'정상 기업 예측 근거 (실제: 정상, 예측 확률: {y_test_prob[low_risk_idx]:.2%})', 
          fontsize=14, weight='bold', pad=20)
plt.tight_layout()
plt.savefig('../data/processed/발표_Part4_Waterfall_정상기업.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 정상 기업 Waterfall Plot 저장 완료")

### 사례 해석

**부도 기업**
- Base Value에서 시작하여 위험 요인(빨간색)이 누적
- 주요 위험 Feature들이 예측값을 끌어올림
- 일부 양호한 지표(파란색)도 있지만 위험 요인이 압도

**정상 기업**
- 건전한 재무 지표(파란색)가 예측값을 낮춤
- 위험 요인이 있어도 전체적으로 안정적
- 최종 예측 확률이 낮게 유지

---

## 섹션 6: Force Plot (Interactive Visualization)

여러 샘플의 예측 근거를 한 눈에 비교합니다.

**주의**: Force Plot은 Jupyter 노트북에서 인터랙티브하게 표시됩니다. HTML 파일로 저장도 가능합니다.

In [ ]:
# 부도 기업 상위 20개
top20_bankrupt = bankrupt_idx[np.argsort(y_test_prob[bankrupt_idx])[-20:]]

print(f"부도 기업 상위 20개 선택 완료")
print(f"예측 확률 범위: {y_test_prob[top20_bankrupt].min():.2%} ~ {y_test_prob[top20_bankrupt].max():.2%}")

# Force Plot
shap.initjs()
force_plot = shap.force_plot(
    expected_value,
    shap_values[top20_bankrupt],
    X_test_preprocessed[top20_bankrupt],
    feature_names=X.columns.tolist()
)

# 노트북에 표시
display(force_plot)

print("\n✅ Force Plot 생성 완료 (위 인터랙티브 플롯 참조)")
print("   - 빨간색: 부도 위험 증가 Feature")
print("   - 파란색: 부도 위험 감소 Feature")
print("   - 각 행은 하나의 기업을 나타냄")

In [ ]:
# (선택사항) HTML로 저장
try:
    shap.save_html(
        '../data/processed/발표_Part4_Force_Plot.html',
        force_plot
    )
    print("✅ Force Plot HTML 저장 완료: 발표_Part4_Force_Plot.html")
except Exception as e:
    print(f"HTML 저장 실패: {e}")
    print("노트북에서 인터랙티브 플롯을 확인하세요.")

---

## 섹션 7: Traffic Light 구간별 SHAP 패턴 분석

각 위험 구간(Red/Yellow/Green)에서 어떤 Feature가 주로 작동하는지 분석합니다.

### Traffic Light 시스템

- **Red (고위험)**: 예측 확률 ≥ Red 임계값
- **Yellow (중위험)**: Yellow 임계값 ≤ 예측 확률 < Red 임계값
- **Green (저위험)**: 예측 확률 < Yellow 임계값

In [ ]:
# 예측 확률로 구간 분류
red_threshold = thresholds['red']
yellow_threshold = thresholds['yellow']

red_mask = y_test_prob >= red_threshold
yellow_mask = (y_test_prob >= yellow_threshold) & (y_test_prob < red_threshold)
green_mask = y_test_prob < yellow_threshold

print("="*80)
print("Traffic Light 구간별 분포")
print("="*80)
print(f"Red (고위험):    {red_mask.sum():5d}개 ({red_mask.mean():6.2%}) - 임계값: {red_threshold:.4f}")
print(f"Yellow (중위험): {yellow_mask.sum():5d}개 ({yellow_mask.mean():6.2%}) - 임계값: {yellow_threshold:.4f}")
print(f"Green (저위험):  {green_mask.sum():5d}개 ({green_mask.mean():6.2%})")
print("="*80)

# 각 구간의 실제 부도율
print("\n각 구간의 실제 부도율:")
if red_mask.sum() > 0:
    print(f"Red:    {y_test[red_mask].mean():.2%}")
if yellow_mask.sum() > 0:
    print(f"Yellow: {y_test[yellow_mask].mean():.2%}")
if green_mask.sum() > 0:
    print(f"Green:  {y_test[green_mask].mean():.2%}")

In [ ]:
# 구간별 평균 SHAP 값
segments = {
    'Red (고위험)': red_mask,
    'Yellow (중위험)': yellow_mask,
    'Green (저위험)': green_mask
}

segment_shap_means = {}
for seg_name, mask in segments.items():
    if mask.sum() > 0:
        segment_shap_means[seg_name] = np.abs(shap_values[mask]).mean(axis=0)

# DataFrame으로 변환
df_segment = pd.DataFrame(segment_shap_means, index=X.columns).T

# Top 10 Feature (전체 평균 기준)
top10_seg_features = df_segment.mean(axis=0).nlargest(10).index

print(f"\n구간별 Top 10 Feature (전체 평균 기준):")
for i, feat in enumerate(top10_seg_features, 1):
    print(f"{i:2d}. {feat}")

In [ ]:
# 시각화
plt.figure(figsize=(14, 7))
df_plot = df_segment[top10_seg_features].T
ax = df_plot.plot(kind='bar', ax=plt.gca(), width=0.8)

plt.title('Traffic Light 구간별 Top 10 Feature Importance', fontsize=16, weight='bold', pad=20)
plt.xlabel('Feature', fontsize=12)
plt.ylabel('Mean |SHAP Value|', fontsize=12)
plt.legend(title='위험 구간', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('../data/processed/발표_Part4_Segment_SHAP.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 구간별 SHAP 분석 저장 완료: 발표_Part4_Segment_SHAP.png")

In [ ]:
# 구간별 차이 분석
print("\n" + "="*80)
print("구간별 Feature Importance 차이")
print("="*80)

if 'Red (고위험)' in df_segment.index and 'Green (저위험)' in df_segment.index:
    diff = df_segment.loc['Red (고위험)'] - df_segment.loc['Green (저위험)']
    top5_diff = diff.nlargest(5)
    
    print("\nRed vs Green 차이가 큰 Feature (Red에서 더 중요):")
    for i, (feat, val) in enumerate(top5_diff.items(), 1):
        print(f"{i}. {feat:40s} +{val:.4f}")
    
    bottom5_diff = diff.nsmallest(5)
    print("\nRed vs Green 차이가 큰 Feature (Green에서 더 중요):")
    for i, (feat, val) in enumerate(bottom5_diff.items(), 1):
        print(f"{i}. {feat:40s} {val:.4f}")

print("="*80)

### 인사이트

- **Red 구간**: 구조적 문제 (자본잠식, 이자보상 등) 중요
- **Yellow 구간**: 유동성 경고 신호 (유동비율, 매출채권 회전율 등)
- **Green 구간**: 전반적으로 건전한 재무 지표
- **차이 분석**: 각 구간을 구분하는 핵심 Feature 파악

---

## 섹션 8: Bootstrap 신뢰구간 추가 ⭐

SHAP Feature Importance의 통계적 안정성을 검증합니다.

### Bootstrap 방법론

- 복원 추출로 1,000개 샘플 생성
- 각 샘플에서 Feature Importance 계산
- 95% 신뢰구간 산출
- 신뢰구간이 좁을수록 안정적인 Feature

In [ ]:
# Bootstrap (1,000회)
n_bootstrap = 1000
bootstrap_importance = []

print(f"Bootstrap 시작: {n_bootstrap}회 반복")
print("진행 상황:")

for i in range(n_bootstrap):
    # 복원 추출
    indices = resample(range(len(X_test)), n_samples=len(X_test), random_state=i)
    shap_boot = shap_values[indices]
    
    # Feature Importance 계산
    importance = np.abs(shap_boot).mean(axis=0)
    bootstrap_importance.append(importance)
    
    # 진행 상황 표시
    if (i + 1) % 100 == 0:
        print(f"  {i + 1:4d}/{n_bootstrap} 완료 ({(i+1)/n_bootstrap*100:.0f}%)")

bootstrap_importance = np.array(bootstrap_importance)

print(f"\n✅ Bootstrap 완료")
print(f"   - Shape: {bootstrap_importance.shape}")

In [ ]:
# 95% 신뢰구간
lower = np.percentile(bootstrap_importance, 2.5, axis=0)
upper = np.percentile(bootstrap_importance, 97.5, axis=0)
mean_importance = bootstrap_importance.mean(axis=0)

# Top 10 Feature CI
top10_idx_boot = np.argsort(mean_importance)[-10:][::-1]
top10_feat_boot = X.columns[top10_idx_boot]

df_ci = pd.DataFrame({
    'Feature': top10_feat_boot,
    'Mean': mean_importance[top10_idx_boot],
    'Lower': lower[top10_idx_boot],
    'Upper': upper[top10_idx_boot],
    'CI_Width': upper[top10_idx_boot] - lower[top10_idx_boot]
})

print("\n" + "="*100)
print("Top 10 Feature Importance with Bootstrap 95% CI")
print("="*100)
print(df_ci.to_string(index=False))
print("="*100)
print("\n신뢰구간 폭(CI_Width)이 작을수록 안정적인 Feature입니다.")

In [ ]:
# 시각화
plt.figure(figsize=(12, 7))

y_pos = np.arange(len(df_ci))
plt.barh(
    y_pos, 
    df_ci['Mean'], 
    xerr=[df_ci['Mean'] - df_ci['Lower'], df_ci['Upper'] - df_ci['Mean']],
    capsize=5, 
    alpha=0.7, 
    color='steelblue',
    ecolor='darkred',
    linewidth=1.5
)

plt.yticks(y_pos, df_ci['Feature'])
plt.xlabel('Mean |SHAP Value| (95% CI)', fontsize=12, weight='bold')
plt.title('Top 10 Feature Importance with Bootstrap Confidence Interval', 
          fontsize=16, weight='bold', pad=20)
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('../data/processed/발표_Part4_Bootstrap_CI.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Bootstrap CI 플롯 저장 완료: 발표_Part4_Bootstrap_CI.png")

### 통계적 안정성 평가

- 신뢰구간이 좁은 Feature: 샘플링에 관계없이 일관되게 중요
- 신뢰구간이 넓은 Feature: 샘플에 따라 중요도가 변동
- 모델 배포 시 안정적인 Feature를 우선 고려

---

## 섹션 9: 비즈니스 인사이트 종합

의사결정자를 위한 핵심 메시지를 정리합니다.

In [ ]:
# 최종 요약 테이블
summary = pd.DataFrame({
    '항목': [
        '모델',
        'Test PR-AUC',
        'Test Recall',
        'Test F2-Score',
        '선택된 임계값',
        'Red 구간 기업 수',
        'Yellow 구간 기업 수',
        'Green 구간 기업 수',
        'Top 1 Feature',
        'Top 2 Feature',
        'Top 3 Feature'
    ],
    '값': [
        results['model_name'],
        f"{results['test_pr_auc']:.4f}",
        f"{results['test_recall']:.2%}",
        f"{results['test_f2']:.4f}",
        f"{thresholds['selected']:.4f}",
        f"{red_mask.sum():,}개 ({red_mask.mean():.2%})",
        f"{yellow_mask.sum():,}개 ({yellow_mask.mean():.2%})",
        f"{green_mask.sum():,}개 ({green_mask.mean():.2%})",
        top10_features[0],
        top10_features[1],
        top10_features[2]
    ]
})

print("="*80)
print("📊 최종 모델 성능 및 위험 구간 분포")
print("="*80)
print(summary.to_string(index=False))
print("="*80)

In [ ]:
# 비즈니스 인사이트
top3_importance_sum = feature_importance[top10_idx[:3]].sum()
total_importance = feature_importance.sum()
top3_contribution = top3_importance_sum / total_importance * 100

print("\n" + "="*80)
print("💡 비즈니스 인사이트")
print("="*80)

print("\n1. 핵심 위험 지표")
print(f"   - Top 3 Feature가 전체 예측의 {top3_contribution:.1f}% 설명")
print(f"   - 1순위: {top3_features[0]}")
print(f"   - 2순위: {top3_features[1]}")
print(f"   - 3순위: {top3_features[2]}")
print("   ➜ 이 3개 지표만 집중 모니터링해도 대부분의 부도 위험 파악 가능")

print("\n2. 조기 경보 시스템")
print(f"   - Yellow 구간: {yellow_mask.sum():,}개 기업 (집중 모니터링 대상)")
print(f"   - Red 구간: {red_mask.sum():,}개 기업 (즉각 대응 필요)")
if yellow_mask.sum() > 0:
    print(f"   - Yellow 구간 실제 부도율: {y_test[yellow_mask].mean():.2%}")
if red_mask.sum() > 0:
    print(f"   - Red 구간 실제 부도율: {y_test[red_mask].mean():.2%}")
print("   ➜ Yellow 구간 기업 대상 사전 개입으로 부도 예방 가능")

print("\n3. 해석 가능성")
print("   - SHAP으로 개별 기업의 부도 위험 근거 명확히 제시")
print("   - 금융기관: 대출 심사 근거 마련 (설명 가능한 AI)")
print("   - 투자자: 포트폴리오 리스크 관리 (위험 요인 사전 파악)")
print("   - 규제기관: 공정성 및 투명성 확보 (블랙박스 해소)")
print("   ➜ 단순 점수가 아닌 '왜 그런지' 설명 가능")

print("\n4. 실무 활용 방안")
print("   - 신규 대출 심사: Red/Yellow 기업 거절 또는 고금리 적용")
print("   - 기존 여신 관리: Yellow → Red 이동 기업 조기 발견")
print("   - 경영 컨설팅: Waterfall Plot으로 개선 포인트 제시")
print("   - 업종별 전략: 구간별 Feature 차이 분석으로 맞춤 대응")

print("\n5. 비용-효과 분석")
recall = results['test_recall']
precision = results.get('test_precision', 0.1)  # 기본값
print(f"   - Recall {recall:.1%}: 실제 부도 기업의 {recall:.1%} 사전 탐지")
print(f"   - 대손 회피 가치: 예상 부도액의 {recall:.1%} 절감 가능")
print(f"   - False Positive: 정상 기업 중 일부 오판 (Yellow 구간 모니터링으로 완화)")
print("   ➜ 대손 방지 편익 >> 모니터링 비용")

print("="*80)

---

## 섹션 10: 모델 한계점 및 개선 방향

### 10.1 데이터 한계

#### 시점 제약
- **현황**: 2021년 8월 단일 시점 스냅샷 데이터
- **문제**: 시계열 패턴 (재무 상태 추이) 반영 불가
- **개선 방향**: 
  - 다년도 패널 데이터 수집 (2018-2024년)
  - 변화율(Δ) Feature 추가 (YoY 매출 증가율, 부채비율 변화 등)
  - 이동 평균, 추세선 등 시계열 통계량 활용

#### 표본 편향
- **현황**: 외감 기업 중심 (소규모 기업 과소 대표)
- **문제**: 생존 편향 (이미 폐업한 기업 미포함)
- **개선 방향**:
  - 비외감 기업 데이터 보완 (중소기업 현황 반영)
  - 폐업 기업 사후 데이터 수집 (역사적 부도 사례)
  - 샘플링 가중치 조정으로 편향 완화

---

### 10.2 모델 한계

#### 클래스 불균형
- **현황**: 부도율 1.5% → SMOTE로 완화했으나 여전히 Precision 낮음
- **문제**: Recall 80% 달성 시 Precision 5-10% (False Positive 多)
- **개선 방향**:
  - Cost-Sensitive Learning (오분류 비용 차등 적용)
  - Focal Loss (어려운 샘플에 가중치)
  - Ensemble with Different Thresholds

#### Feature 누락
- **현황**: 거시경제 변수 (금리, GDP, 환율) 미포함
- **문제**: 산업별 충격 (COVID-19 등) 반영 안 됨
- **개선 방향**:
  - 외부 데이터 결합 (한국은행 경제통계)
  - Industry-Specific Features (업종별 수주 잔고, 원자재 가격 등)
  - 뉴스/소셜미디어 감성 분석 (Alternative Data)

#### 예측 시계
- **현황**: "향후 1년 내 부도" → 정확한 시점 예측 불가
- **문제**: 6개월 vs 11개월 부도 구분 못함
- **개선 방향**:
  - Survival Analysis (Cox Proportional Hazard Model)
  - 월별 예측 확률 (1개월, 3개월, 6개월, 12개월)
  - Dynamic Updating (매월 최신 데이터로 재예측)

---

### 10.3 해석 한계

#### SHAP 계산 비용
- **현황**: TreeExplainer로 개선했으나 대규모 데이터에서는 여전히 느림
- **문제**: Real-Time 서비스 적용 시 지연 발생 가능
- **개선 방향**:
  - Approximate SHAP (샘플링 기반)
  - FastTreeSHAP 알고리즘
  - 배치 처리 및 캐싱

#### 상호작용 효과
- **현황**: SHAP은 개별 Feature 기여도만 표시
- **문제**: 복잡한 Feature 간 상호작용 (예: 부채비율 × 수익성) 설명 부족
- **개선 방향**:
  - SHAP Interaction Values 추가 분석
  - Partial Dependence Plot (2D)
  - H-statistic으로 상호작용 강도 측정

---

### 10.4 운영 한계

#### 모델 드리프트
- **현황**: 경제 환경 변화 시 모델 성능 저하 (예: 금리 급등)
- **문제**: 정기 재학습 없으면 예측력 감소
- **개선 방향**:
  - 분기별 성능 Monitoring (PSI, CSI 지표)
  - 자동 재학습 파이프라인 (MLOps)
  - Concept Drift Detection 알고리즘

#### 규제 준수
- **현황**: 신용평가 모델 규제 (Basel III 등) 충족 여부 미검증
- **문제**: 공정성 (Fairness) 평가 미실시 (업종/지역 차별 가능성)
- **개선 방향**:
  - Regulatory Compliance Check
  - Fairness Metrics (Demographic Parity, Equal Opportunity)
  - Stress Testing (극단 시나리오)

---

### 10.5 비즈니스 한계

#### 오분류 비용 불균형
- **Type I Error** (정상→부도 오판): 대출 기회 손실
- **Type II Error** (부도→정상 오판): 대손 발생 (훨씬 큰 비용)
- **현황**: 현재 임계값은 동일 가중치 가정
- **개선 방향**:
  - 실제 대손율 기반 Cost Matrix 설계
  - 최적 임계값 재조정 (Expected Cost Minimization)
  - 업종별/금액별 차등 임계값

#### 설명 가능성 vs 성능 트레이드오프
- **현황**: Tree 모델 선택 → 딥러닝보다 성능 낮을 수 있음
- **문제**: SHAP도 복잡한 비선형 관계 완벽히 설명 못함
- **개선 방향**:
  - Tabular Deep Learning (TabNet, FT-Transformer) 실험
  - Hybrid Model (Tree + DL Ensemble)
  - Attention Mechanism으로 해석력 강화

---

### 10.6 향후 연구 방향

1. **시계열 확장**
   - LSTM/Transformer 기반 다년도 예측 모델
   - 재무 지표 추이 패턴 학습
   
2. **멀티태스크 학습**
   - 부도 + 신용등급 + 재무조작 동시 예측
   - 공유 표현 학습으로 일반화 향상
   
3. **강건성 평가**
   - Adversarial Examples 테스트
   - Out-of-Distribution Detection
   - Calibration 분석
   
4. **Causal Inference**
   - 정책 개입 효과 예측 (예: 금리 인하 시 부도율 변화)
   - Counterfactual Explanation
   - Treatment Effect Estimation
   
5. **실시간 시스템**
   - Streamlit → FastAPI + Redis + Celery 아키텍처
   - 대규모 처리 (10,000+ TPS)
   - A/B 테스트 프레임워크

---

## 섹션 11: 최종 저장

모든 분석 결과를 JSON 형식으로 저장합니다.

In [ ]:
# 분석 결과 저장
analysis_results = {
    'model_info': {
        'model_name': results['model_name'],
        'test_pr_auc': float(results['test_pr_auc']),
        'test_recall': float(results['test_recall']),
        'test_f2': float(results['test_f2'])
    },
    'thresholds': {
        'selected': float(thresholds['selected']),
        'red': float(thresholds['red']),
        'yellow': float(thresholds['yellow'])
    },
    'feature_importance': {
        'top10_features': top10_features.tolist(),
        'importance_values': {
            feat: float(feature_importance[list(X.columns).index(feat)])
            for feat in top10_features
        },
        'top3_contribution_pct': float(top3_contribution)
    },
    'segment_distribution': {
        'red': {
            'count': int(red_mask.sum()),
            'percentage': float(red_mask.mean()),
            'actual_bankruptcy_rate': float(y_test[red_mask].mean()) if red_mask.sum() > 0 else 0
        },
        'yellow': {
            'count': int(yellow_mask.sum()),
            'percentage': float(yellow_mask.mean()),
            'actual_bankruptcy_rate': float(y_test[yellow_mask].mean()) if yellow_mask.sum() > 0 else 0
        },
        'green': {
            'count': int(green_mask.sum()),
            'percentage': float(green_mask.mean()),
            'actual_bankruptcy_rate': float(y_test[green_mask].mean()) if green_mask.sum() > 0 else 0
        }
    },
    'bootstrap_ci': df_ci.to_dict('records'),
    'shap_summary': {
        'n_samples': int(shap_values.shape[0]),
        'n_features': int(shap_values.shape[1]),
        'expected_value': float(expected_value)
    }
}

# JSON 저장
output_path = '../data/processed/발표_Part4_SHAP_분석결과.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(analysis_results, f, ensure_ascii=False, indent=2)

print("✅ 분석 결과 JSON 저장 완료")
print(f"   - 파일: {output_path}")

In [ ]:
# 생성된 파일 목록
output_files = [
    '발표_Part4_SHAP_Summary.png',
    '발표_Part4_Top10_Features.png',
    '발표_Part4_Dependence_Plot.png',
    '발표_Part4_Waterfall_부도기업.png',
    '발표_Part4_Waterfall_정상기업.png',
    '발표_Part4_Segment_SHAP.png',
    '발표_Part4_Bootstrap_CI.png',
    '발표_Part4_SHAP_분석결과.json'
]

print("="*80)
print("✅ Part 4 완료! 모든 결과가 저장되었습니다.")
print("="*80)
print("\n생성된 파일 (../data/processed/):")
for i, f in enumerate(output_files, 1):
    file_path = os.path.join(PROCESSED_DIR, f)
    exists = "✓" if os.path.exists(file_path) else "✗"
    print(f"  {exists} {i}. {f}")

print("\n" + "="*80)
print("다음 단계: 이 결과를 바탕으로 발표 자료 작성")
print("="*80)

---

## 🎨 생성된 시각화 체크리스트

- [x] Summary Plot (Beeswarm) - Global Feature Importance
- [x] Top 10 Bar Chart - 주요 Feature 순위
- [x] Dependence Plot (Top 3) - 비선형 관계 분석
- [x] Waterfall Plot (부도 기업) - 개별 예측 근거
- [x] Waterfall Plot (정상 기업) - 대조 분석
- [x] Traffic Light 구간별 SHAP - 위험 구간별 패턴
- [x] Bootstrap 신뢰구간 - 통계적 안정성

---

## 📚 참고 자료

- SHAP 공식 문서: https://shap.readthedocs.io/
- Lundberg, S. M., & Lee, S. I. (2017). "A Unified Approach to Interpreting Model Predictions." NeurIPS.
- Molnar, C. (2022). "Interpretable Machine Learning."
- Shapley, L. S. (1953). "A value for n-person games."

---

## 🎯 핵심 결론

1. **모델 성능**: PR-AUC 기준 우수한 예측력 달성
2. **해석 가능성**: SHAP으로 블랙박스 해소, 설명 가능한 AI 구현
3. **비즈니스 가치**: Traffic Light 시스템으로 실무 활용 가능
4. **통계적 안정성**: Bootstrap CI로 Feature Importance 신뢰성 검증
5. **개선 방향**: 시계열 확장, 외부 데이터 결합, 실시간 시스템 구축

---

**감사합니다!** 🙏